# Задание
Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД.

Магазины можно выбрать свои.

Главный критерий выбора: динамически загружаемые товары.

# Решение

## Парсинг

Блок импорта

In [74]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pymongo import MongoClient
import json

headless режим поддерживается только в chrome, но сохраню на всякий случай

In [3]:
# from selenium.webdriver.chrome.options import Options
# options = Options()
# options.add_argument('--headless')

инициализация драйвера и установка размера окна, т.к. нужных элементов при меньшем размере нет. в консоли предварительно нужно ввести следующие команды:
```safaridriver --enable
safaridriver --port 0```

In [56]:
driver = webdriver.Safari()
driver.set_window_size(1500, 620)

открываем mvideo.ru

In [57]:
driver.get('https://www.mvideo.ru')

прокрутим карусель до последнего окна, чтобы все элементы гарантированно подгрузились. `hits_last_btn.click()` почему-то не работает.

In [59]:
hits_div = driver.find_elements_by_css_selector('div.sel-hits-block')[0]
hits_last_btn = hits_div.find_elements_by_css_selector('div.carousel-paging > a')[-1]
driver.execute_script("arguments[0].click();", hits_last_btn)

парсим элементы

In [73]:
goods = hits_div.find_elements_by_css_selector('ul.accessories-product-list > li div.c-product-tile-picture__holder > a[data-product-info]')
json_raw = [e.get_attribute('data-product-info') for e in goods]
json_parsed = [json.loads(e) for e in json_raw]
urls = [e.get_attribute('href') for e in goods]
total = list(zip(urls, json_parsed))
print('Всего:', len(total))
print('Пример:\n', total[0])

Всего: 20
Пример:
 ('https://www.mvideo.ru/products/naushniki-bluetooth-qub-qtwe1-black-50125744', {'productPriceLocal': '2990.00', 'productId': '50125744', 'productName': 'Наушники Bluetooth QUB QTWE1 Black', 'productCategoryId': 'cat2_cis_0000000728', 'productCategoryName': 'Наушники', 'productVendorName': 'QUB', 'productGroupId': 'cat1_cis_0000000155', 'Location': 'block5260655', 'eventPosition': 17})


## Сохраняем результаты

стартуем образ с mongodb

In [78]:
!docker start scrapy_mongo

scrapy_mongo


записываем результат в бд

In [79]:
client = MongoClient('localhost', 27017)
db = client.selenium
collection = db['mvideo']
for product in total:
    collection.update_one({'_id': product[0]}, {'$set': product[1]}, upsert=True)

## Завершаем работу

останавливаем mongodb

In [80]:
!docker stop scrapy_mongo

scrapy_mongo


завершаем работу драйвера selenium

In [81]:
driver.quit()